In [61]:

import pickle
import sys
from bisect import bisect_left,bisect_right

In [2]:
folder_location = "/home/himanshu/Desktop/inverted_index/1gig/"

In [3]:
#importing the saved dictionary
pickle_in = open("word_location_dictionary","rb")
word_file_dict = pickle.load(pickle_in)

In [4]:
(filename, line_no) = word_file_dict['2380']

In [55]:
def intersection(list1,list2):
    list3=[value for value in list2 if value in list1]
    return list3

In [5]:
#how 
def generate_line(filename, line_no):
    with open(folder_location + filename) as file:
        i=0
        while i < line_no:
            next(file)
            i+=1
        line = file.readline()
    return line
    
    

In [6]:
# print(sys.getsizeof(line))

NameError: name 'line' is not defined

In [7]:
def extract_doc_list(line):
    i=0
    j=0
    while line[i] is not '[':
        i+=1
    j=i+1
    
    while line[j] is not ']':
        j+=1
    
    doc_str = line[i+1:j].split(", ")
    doc_as_int = [int(docid) for docid in doc_str]
    
    return doc_as_int
    

In [9]:
def extract_pos_list(line):
    i =0
    while line[i] is not ']':
        i+=1
    while line[i] is not '[':
        i+=1
    j=i+1
    while line[j] is not ')':
        j+=1
    
    pos_list_str = line[i+2:j]
    pos_list_sep = pos_list_str.split("], [")
    pos_list_sep = ['['+positions+']' for positions in pos_list_sep ]
    
    pos_list = [extract_doc_list(each) for each in pos_list_sep]
    
    return pos_list
        

In [75]:
pos_list_sep = extract_pos_list(line)

In [ ]:
print(pos_list_sep)

In [83]:
position_list_int = [extract_doc_list(each) for each in pos_list_sep]

In [ ]:
position_list_int

In [85]:
doc_posting_dict = {doc:pos_list for (doc,pos_list) in zip(docid_list,position_list_int)}

In [86]:
doc_posting_dict[25947]

[97, 100]

In [15]:
def doc_post_gen(word):
    (filename, line_no) = word_file_dict[word]
    line = generate_line(filename,line_no)
    docid_list = extract_doc_list(line)
    pos_list = extract_pos_list(line)
    
    doc_posting_dict = {doc:pos_list for (doc,pos_list) in zip(docid_list,pos_list)}
    return (docid_list, doc_posting_dict)
    
    
    
    
    

In [16]:
doc_list, doc_posting_dict = doc_post_gen('2380')

## Some constants and how to evaluate
We will require->
total_docs
dl:document length
adl:average document length
average_word_count - on average how many times a unique word appears in the document-
given by dl/unique_words_in_doc

How to evaluate->
split in the form (d,w,1,1,1)
make (d,w) to be a key and then just add total occurences of this key
after this remove w from key and just count total rows of d or append a 1 and sum
now reduce by key to get total unique words in the documents.
save this to a python dictionary.
doc -> total_words, unique_wprds
now just sum over all occurences to get total words in all documents and also count total number of rows to get adl

In [66]:
def ow_occurences(list1,list2, bin_flag = False):
    occurences = 0
    for each in list1:
        if bin_flag is False:
            upper_bound = each+15
            lower_bound = each-15
        else:
            upper_bound = each+1
            lower_bound = each
        
        list2_upper = bisect_left(list2, upper_bound)
        if list2_upper==len(list2) or list2[list2_upper] is not upper_bound:
            list2_upper-=1
#         print("for {} the upper bound is {}".format(each,list2_upper))
        
        list2_lower = bisect_left(list2, lower_bound)
#         print("for {} the lower bound is {}".format(each,list2_lower))
        
        occur_indv = list2_upper - list2_lower+1
#         print(occur_indv)
        
        occurences += (list2_upper - list2_lower+1)
        
    return occurences

            

In [67]:
#DEFINE THIS CONSTANT
total_documents = 50656000 #this is something random
N=total_documents
def s1(query):
    for word in query:
        doc_list, doc_posting_dict = doc_post_gen(word)
        
        idf_w = log(N/len(doc_list))
        
        for each doc in doc_list:
            occurence = len(doc_posting_dict[doc])
            #MAKE SURE THAT BELOW DICTIONARY IS CREATED
            doc_length,avg_word_count = doc_param[doc]
            #MAY TAKE AVERAGE WORD AS AVERAGE UNIQUE WORDS IN THE DOCUMENT
            TFF
            
        

SyntaxError: invalid syntax (<ipython-input-67-98e3fc40c0b0>, line 10)

In [83]:
def query_firer(query):
    doc_query = []
    doc_pos_dicts = []
#     print("Checkpoint1")
    for word in query:
        docs, doc_post = doc_post_gen(word)
        doc_query.append(docs)
        doc_pos_dicts.append(doc_post)
#     print("Reached here")
#     print("length of query is {}".format(len(query)))
    for i in range(len(query)):
#         print("i value is {}".format(i))
        if i < (len(query)-1):
            j = i+1
#             print("j value is {}".format(j))
#             print("total docs for word 1 are {}".format(len(doc_query[i])))
#             print("total docs for word 2 are {}".format(len(doc_query[j])))
# #             print("docs for word1 are {}".format(doc_query[i]))
#             print("docs for word2 are {}".format(doc_query[j]))
            common_docs = intersection(doc_query[i],doc_query[j])
#             print("list of common docs is {}".format(common_docs))
            
            for doc in common_docs:
                successive_occurence = ow_occurences(doc_pos_dicts[i][doc], doc_pos_dicts[j][doc], True)
                print("Successive occurence for doc-{} are {}".format(doc,successive_occurence))
                
                window_ocs = ow_occurences(doc_pos_dicts[i][doc], doc_pos_dicts[j][doc])
                print("Total window occurences for doc-{} are {}".format(doc,window_ocs))
    
#             print(common_docs)

In [84]:
query_firer(['134', '217'])

Successive occurence for doc-418 are 0
Total window occurences for doc-418 are 0
Successive occurence for doc-782 are 0
Total window occurences for doc-782 are 1
Successive occurence for doc-6999 are 0
Total window occurences for doc-6999 are 0
Successive occurence for doc-13838 are 0
Total window occurences for doc-13838 are 0
Successive occurence for doc-15458 are 0
Total window occurences for doc-15458 are 0
Successive occurence for doc-16844 are 0
Total window occurences for doc-16844 are 1
Successive occurence for doc-19999 are 0
Total window occurences for doc-19999 are 0
Successive occurence for doc-23972 are 0
Total window occurences for doc-23972 are 0
Successive occurence for doc-25785 are 0
Total window occurences for doc-25785 are 0
Successive occurence for doc-34406 are 0
Total window occurences for doc-34406 are 1
Successive occurence for doc-102686 are 0
Total window occurences for doc-102686 are 0
Successive occurence for doc-103201 are 0
Total window occurences for doc

Total window occurences for doc-8827297 are 0
Successive occurence for doc-8827372 are 0
Total window occurences for doc-8827372 are 0
Successive occurence for doc-8829316 are 0
Total window occurences for doc-8829316 are 1
Successive occurence for doc-8829767 are 0
Total window occurences for doc-8829767 are 0
Successive occurence for doc-8833407 are 0
Total window occurences for doc-8833407 are 0
Successive occurence for doc-8835058 are 0
Total window occurences for doc-8835058 are 0
Successive occurence for doc-8903173 are 0
Total window occurences for doc-8903173 are 0
Successive occurence for doc-8905030 are 0
Total window occurences for doc-8905030 are 0
Successive occurence for doc-8915864 are 0
Total window occurences for doc-8915864 are 0
Successive occurence for doc-8921155 are 0
Total window occurences for doc-8921155 are 0
Successive occurence for doc-8927027 are 0
Total window occurences for doc-8927027 are 0
Successive occurence for doc-8929755 are 0
Total window occurenc

Total window occurences for doc-23525444 are 0
Successive occurence for doc-23528117 are 0
Total window occurences for doc-23528117 are 0
Successive occurence for doc-23531812 are 0
Total window occurences for doc-23531812 are 0
Successive occurence for doc-23532046 are 0
Total window occurences for doc-23532046 are 0
Successive occurence for doc-23601647 are 0
Total window occurences for doc-23601647 are 0
Successive occurence for doc-23601980 are 0
Total window occurences for doc-23601980 are 0
Successive occurence for doc-23604622 are 0
Total window occurences for doc-23604622 are 0
Successive occurence for doc-23605544 are 0
Total window occurences for doc-23605544 are 0
Successive occurence for doc-23605964 are 0
Total window occurences for doc-23605964 are 0
Successive occurence for doc-23605995 are 0
Total window occurences for doc-23605995 are 0
Successive occurence for doc-23619167 are 0
Total window occurences for doc-23619167 are 0
Successive occurence for doc-23619404 are 0

Total window occurences for doc-39223628 are 0
Successive occurence for doc-39228254 are 0
Total window occurences for doc-39228254 are 0
Successive occurence for doc-39229145 are 0
Total window occurences for doc-39229145 are 1
Successive occurence for doc-39229307 are 0
Total window occurences for doc-39229307 are 0
Successive occurence for doc-39302273 are 0
Total window occurences for doc-39302273 are 0
Successive occurence for doc-39308565 are 0
Total window occurences for doc-39308565 are 0
Successive occurence for doc-39324796 are 0
Total window occurences for doc-39324796 are 0
Successive occurence for doc-39325236 are 0
Total window occurences for doc-39325236 are 2
Successive occurence for doc-39402632 are 0
Total window occurences for doc-39402632 are 0
Successive occurence for doc-39406034 are 0
Total window occurences for doc-39406034 are 0
Successive occurence for doc-39406560 are 0
Total window occurences for doc-39406560 are 0
Successive occurence for doc-39412389 are 0

Total window occurences for doc-50331506 are 0
Successive occurence for doc-50331988 are 0
Total window occurences for doc-50331988 are 0
Successive occurence for doc-50402163 are 0
Total window occurences for doc-50402163 are 1
Successive occurence for doc-50415325 are 0
Total window occurences for doc-50415325 are 0
Successive occurence for doc-50423344 are 0
Total window occurences for doc-50423344 are 4
Successive occurence for doc-50423506 are 0
Total window occurences for doc-50423506 are 0
Successive occurence for doc-50425546 are 0
Total window occurences for doc-50425546 are 0
Successive occurence for doc-50427210 are 0
Total window occurences for doc-50427210 are 0
Successive occurence for doc-50427328 are 0
Total window occurences for doc-50427328 are 0
Successive occurence for doc-50427359 are 0
Total window occurences for doc-50427359 are 0
Successive occurence for doc-50428101 are 0
Total window occurences for doc-50428101 are 0
Successive occurence for doc-50500276 are 0

...

Total window occurences for doc-78626198 are 0
Successive occurence for doc-78626242 are 0
Total window occurences for doc-78626242 are 0
Successive occurence for doc-78629892 are 0
Total window occurences for doc-78629892 are 0
Successive occurence for doc-78631093 are 0
Total window occurences for doc-78631093 are 1
Successive occurence for doc-78636731 are 0
Total window occurences for doc-78636731 are 0
Successive occurence for doc-78639080 are 0
Total window occurences for doc-78639080 are 0
Successive occurence for doc-78703336 are 0
Total window occurences for doc-78703336 are 0
Successive occurence for doc-78703787 are 0
Total window occurences for doc-78703787 are 0
Successive occurence for doc-78703862 are 0
Total window occurences for doc-78703862 are 0
Successive occurence for doc-78705031 are 0
Total window occurences for doc-78705031 are 0
Successive occurence for doc-78705451 are 0
Total window occurences for doc-78705451 are 4
Successive occurence for doc-78705482 are 0

Total window occurences for doc-87725163 are 2
Successive occurence for doc-87726829 are 0
Total window occurences for doc-87726829 are 0
Successive occurence for doc-87727589 are 0
Total window occurences for doc-87727589 are 0
Successive occurence for doc-87727602 are 0
Total window occurences for doc-87727602 are 0
Successive occurence for doc-87728944 are 0
Total window occurences for doc-87728944 are 9
Successive occurence for doc-87729091 are 0
Total window occurences for doc-87729091 are 1
Successive occurence for doc-87731098 are 0
Total window occurences for doc-87731098 are 0
Successive occurence for doc-87731724 are 0
Total window occurences for doc-87731724 are 0
Successive occurence for doc-87732808 are 0
Total window occurences for doc-87732808 are 0
Successive occurence for doc-87804969 are 0
Total window occurences for doc-87804969 are 0
Successive occurence for doc-87807524 are 0
Total window occurences for doc-87807524 are 0
Successive occurence for doc-87807944 are 0

Successive occurence for doc-99728615 are 0
Total window occurences for doc-99728615 are 0
Successive occurence for doc-99731071 are 0
Total window occurences for doc-99731071 are 0
Successive occurence for doc-99735217 are 0
Total window occurences for doc-99735217 are 0
Successive occurence for doc-99801370 are 0
Total window occurences for doc-99801370 are 0
Successive occurence for doc-99804625 are 0
Total window occurences for doc-99804625 are 0
Successive occurence for doc-99807824 are 0
Total window occurences for doc-99807824 are 0
Successive occurence for doc-99808027 are 1
Total window occurences for doc-99808027 are 2
Successive occurence for doc-99810442 are 0
Total window occurences for doc-99810442 are 0
Successive occurence for doc-99811140 are 0
Total window occurences for doc-99811140 are 0
Successive occurence for doc-99812224 are 0
Total window occurences for doc-99812224 are 0
Successive occurence for doc-99815812 are 0
Total window occurences for doc-99815812 are 0

Total window occurences for doc-108707967 are 0
Successive occurence for doc-108714031 are 0
Total window occurences for doc-108714031 are 0
Successive occurence for doc-108714675 are 0
Total window occurences for doc-108714675 are 0
Successive occurence for doc-108718703 are 0
Total window occurences for doc-108718703 are 1
Successive occurence for doc-108719270 are 0
Total window occurences for doc-108719270 are 0
Successive occurence for doc-108720461 are 0
Total window occurences for doc-108720461 are 0
Successive occurence for doc-108721934 are 0
Total window occurences for doc-108721934 are 0
Successive occurence for doc-108722081 are 0
Total window occurences for doc-108722081 are 0
Successive occurence for doc-108722694 are 0
Total window occurences for doc-108722694 are 0
Successive occurence for doc-108802103 are 0
Total window occurences for doc-108802103 are 0
Successive occurence for doc-108808846 are 0
Total window occurences for doc-108808846 are 0
Successive occurence f

Successive occurence for doc-118718627 are 0
Total window occurences for doc-118718627 are 0
Successive occurence for doc-118721771 are 0
Total window occurences for doc-118721771 are 0
Successive occurence for doc-118726033 are 0
Total window occurences for doc-118726033 are 0
Successive occurence for doc-118800003 are 0
Total window occurences for doc-118800003 are 0
Successive occurence for doc-118806908 are 0
Total window occurences for doc-118806908 are 0
Successive occurence for doc-118807024 are 0
Total window occurences for doc-118807024 are 0
Successive occurence for doc-118809590 are 0
Total window occurences for doc-118809590 are 0
Successive occurence for doc-118811146 are 0
Total window occurences for doc-118811146 are 0
Successive occurence for doc-118813616 are 0
Total window occurences for doc-118813616 are 0
Successive occurence for doc-118816385 are 0
Total window occurences for doc-118816385 are 0
Successive occurence for doc-118818167 are 0
Total window occurences f